In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-train-and-test-data-for-local-validation/'

df_train = pd.read_parquet(INPUT_DIR+'train.parquet')
df_val = pd.read_parquet(INPUT_DIR+'test.parquet')
#test_labels = pd.read_parquet(INPUT_DIR+'test_labels.parquet')
print(len(df_train))
print(len(df_val))

163955180
7683577


In [ ]:
#data_dir = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-train-and-test-data-for-local-validation'

#df_train = pd.read_parquet(INPUT_DIR+'/train.parquet')
#df_val = pd.read_parquet(INPUT_DIR+'/test.parquet')

In [ ]:
df_val_part = df_val.iloc[:10]
df_val_part.columns

Index(['session', 'aid', 'ts', 'type'], dtype='object')

# step1: valid Aのsessionのごとにitem候補生成

In [ ]:
'''
# step1
## sessionでまとめたデータを入力し、candidateとなるaidのlist返却
def generate_candidate(df):
  df = df.
    # 入力 ある

    return [1, 2, 3, 4]
'''

'\n# step1\n## sessionでまとめたデータを入力し、candidateとなるaidのlist返却\ndef generate_candidate(df):\n  df = df.\n    # 入力 ある\n\n    return [1, 2, 3, 4]\n'

covis matrix読み込み

In [ ]:
%%time
VER = 6
DISK_PIECES = 4

def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# LOAD THREE CO-VISITATION MATRICES
top_20_clicks = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_20_clicks_v{VER}_{k}.pqt') ) )

CPU times: user 42.2 s, sys: 2.64 s, total: 44.8 s
Wall time: 45.1 s


In [ ]:
from collections import Counter

def generate_candidate(df):
  aids = df.aid.tolist()
  candidates = []
  for k in aids:
        if k in top_20_clicks.keys():
            candidates.extend(top_20_clicks[k])
        else:
            pass
  output_candidateb = [k for k,v in collections.Counter(candidates).most_common(20)]
  
  return output_candidateb

In [ ]:
df_val

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


In [ ]:
%%time

candidates = df_val.groupby('session').apply(generate_candidate).explode()
candidates

CPU times: user 3min 7s, sys: 2.44 s, total: 3min 9s
Wall time: 3min 8s


session
11098528     588923
11098528    1732105
11098528     571762
11098528     884502
11098528    1157882
             ...   
12899778      13568
12899778    1097813
12899778    1102546
12899778    1068655
12899778    1495496
Length: 35715483, dtype: object

In [ ]:
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

,session,item
0,11098528,588923
1,11098528,1732105
2,11098528,571762
3,11098528,884502
4,11098528,1157882
...,...,...
35715478,12899778,13568
35715479,12899778,1097813
35715480,12899778,1102546
35715481,12899778,1068655


In [ ]:
candidates[:40]

,session,item
0,11098528,588923
1,11098528,1732105
2,11098528,571762
3,11098528,884502
4,11098528,1157882
5,11098528,876129
6,11098528,1182614
7,11098528,1790438
8,11098528,307904
9,11098528,231487


# step2 : train + valid A のitem特徴量生成

In [ ]:

item_features = pd.merge(df_train, df_val).groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
item_features.to_parquet('item_features.pqt')
item_features

,item_item_count,item_user_count,item_by_count
aid,,,


# step3 : valid A の user特徴量生成

In [ ]:

user_features = df_val.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
user_features.to_parquet('user_features.pqt')
user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
11098528,1,1,0.000000
11098529,1,1,0.000000
11098530,6,2,0.166667
11098531,24,11,0.333333
11098532,2,2,0.000000
...,...,...,...
12899774,1,1,0.000000
12899775,1,1,0.000000
12899776,1,1,0.000000


# step4 : valid A の user × item 特徴量生成？

# step5 : step1に step2,3を追加

In [ ]:

candidates = candidates.rename(columns={'session':'user'})

item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_features.pqt')
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)
candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio
0,11098528,588923,-1.0,-1.0,-1.0,1,1,0.0
1,11098528,1732105,-1.0,-1.0,-1.0,1,1,0.0
2,11098528,571762,-1.0,-1.0,-1.0,1,1,0.0
3,11098528,884502,-1.0,-1.0,-1.0,1,1,0.0
4,11098528,1157882,-1.0,-1.0,-1.0,1,1,0.0
...,...,...,...,...,...,...,...,...
35715478,12899778,13568,-1.0,-1.0,-1.0,1,1,0.0
35715479,12899778,1097813,-1.0,-1.0,-1.0,1,1,0.0
35715480,12899778,1102546,-1.0,-1.0,-1.0,1,1,0.0
35715481,12899778,1068655,-1.0,-1.0,-1.0,1,1,0.0


# step6 : 学習できる形にするgt:0,gt:1

In [ ]:
tar = pd.read_parquet(INPUT_DIR+'test_labels.parquet')
tar = tar.loc[ tar['type']=='clicks' ]
tar['item'] = tar['ground_truth'].explode().astype('int32')
tar['click'] = 1

tar = tar.rename(columns={'session':'user'})
tar

,user,type,ground_truth,item,click
0,11098528,clicks,[1679529],1679529,1
3,11098529,clicks,[1105029],1105029,1
6,11098532,clicks,[1596491],1596491,1
7,11098533,clicks,[1417450],1417450,1
10,11098534,clicks,[908024],908024,1
...,...,...,...,...,...
2212687,12899774,clicks,[1399483],1399483,1
2212688,12899775,clicks,[1760714],1760714,1
2212689,12899776,clicks,[1737908],1737908,1
2212690,12899777,clicks,[384045],384045,1


In [ ]:
tar

,user,type,ground_truth,item,click
0,11098528,clicks,[1679529],1679529,1
3,11098529,clicks,[1105029],1105029,1
6,11098532,clicks,[1596491],1596491,1
7,11098533,clicks,[1417450],1417450,1
10,11098534,clicks,[908024],908024,1
...,...,...,...,...,...
2212687,12899774,clicks,[1399483],1399483,1
2212688,12899775,clicks,[1760714],1760714,1
2212689,12899776,clicks,[1737908],1737908,1
2212690,12899777,clicks,[384045],384045,1


In [ ]:
tar = tar[['user', 'item', 'click']]
tar

,user,item,click
0,11098528,1679529,1
3,11098529,1105029,1
6,11098532,1596491,1
7,11098533,1417450,1
10,11098534,908024,1
...,...,...,...
2212687,12899774,1399483,1
2212688,12899775,1760714,1
2212689,12899776,1737908,1
2212690,12899777,384045,1


In [ ]:
candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)

In [ ]:
candidates

,user,item,item_item_count,item_user_count,item_by_count,user_user_count,user_item_count,user_buy_ratio,click
0,11098528,588923,-1.0,-1.0,-1.0,1,1,0.0,0.0
1,11098528,1732105,-1.0,-1.0,-1.0,1,1,0.0,0.0
2,11098528,571762,-1.0,-1.0,-1.0,1,1,0.0,0.0
3,11098528,884502,-1.0,-1.0,-1.0,1,1,0.0,0.0
4,11098528,1157882,-1.0,-1.0,-1.0,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
35715478,12899778,13568,-1.0,-1.0,-1.0,1,1,0.0,0.0
35715479,12899778,1097813,-1.0,-1.0,-1.0,1,1,0.0,0.0
35715480,12899778,1102546,-1.0,-1.0,-1.0,1,1,0.0,0.0
35715481,12899778,1068655,-1.0,-1.0,-1.0,1,1,0.0,0.0


# Training

In [ ]:
FEATURES = list(candidates.columns[2:8])
FEATURES

['item_item_count',
 'item_user_count',
 'item_by_count',
 'user_user_count',
 'user_item_count',
 'user_buy_ratio']

In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GroupKFold

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['click'], groups=candidates['user'] )):
    print(f'=====fold : {fold}=====')

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, 'click']
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, 'click']

    #ラデック　ranker公開コードパラメータ　https://www.kaggle.com/code/radek1/polars-proof-of-concept-lgbm-ranker
    model = lgb.LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        boosting_type="dart",
        n_estimators=20,
        importance_type='gain',
    )

    model.fit(
        X_train,
        y_train,
        group=candidates.loc[train_idx,:].groupby('user')['item'].agg('count').values ,
        eval_set=[(X_valid, y_valid)],
        eval_group=[candidates.loc[valid_idx,:].groupby('user')['item'].agg('count').values ]
    )

    #model.save_model(f'LGBM_fold{fold}_click.xgb')

=====fold : 0=====
[1]	valid_0's ndcg@1: 0.792383
[2]	valid_0's ndcg@1: 0.792383
[3]	valid_0's ndcg@1: 0.792383
[4]	valid_0's ndcg@1: 0.792383
[5]	valid_0's ndcg@1: 0.792383
[6]	valid_0's ndcg@1: 0.792383
[7]	valid_0's ndcg@1: 0.792383
[8]	valid_0's ndcg@1: 0.792383
[9]	valid_0's ndcg@1: 0.792383
[10]	valid_0's ndcg@1: 0.792383
[11]	valid_0's ndcg@1: 0.792383
[12]	valid_0's ndcg@1: 0.792383
[13]	valid_0's ndcg@1: 0.792383
[14]	valid_0's ndcg@1: 0.792383
[15]	valid_0's ndcg@1: 0.792383
[16]	valid_0's ndcg@1: 0.792383
[17]	valid_0's ndcg@1: 0.792383
[18]	valid_0's ndcg@1: 0.792383
[19]	valid_0's ndcg@1: 0.792383
[20]	valid_0's ndcg@1: 0.792383
=====fold : 1=====
[1]	valid_0's ndcg@1: 0.792658
[2]	valid_0's ndcg@1: 0.792658
[3]	valid_0's ndcg@1: 0.792658
[4]	valid_0's ndcg@1: 0.792658
[5]	valid_0's ndcg@1: 0.792658
[6]	valid_0's ndcg@1: 0.792658
[7]	valid_0's ndcg@1: 0.792658
[8]	valid_0's ndcg@1: 0.792658
[9]	valid_0's ndcg@1: 0.792658
[10]	valid_0's ndcg@1: 0.792658
[11]	valid_0's ndcg@

In [ ]:
stop

NameError: ignored

In [ ]:
candidates.loc[train_idx,:].groupby('user')['item'].agg('count').values

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['click'], groups=candidates['user'] )):

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, 'click']
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, 'click']

    # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
    dtrain = xgb.DMatrix(X_train, y_train, group=candidates.loc[train_idx,:].groupby('user')['item'].agg('count').values ) 
    dvalid = xgb.DMatrix(X_valid, y_valid, group=candidates.loc[valid_idx,:].groupby('user')['item'].agg('count').values ) 

    xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
    model = xgb.train(xgb_parms, 
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'XGB_fold{fold}_click.xgb')